In [1]:
import keras
from keras import layers

from imgaug.augmentables.kps import KeypointsOnImage
from imgaug.augmentables.kps import Keypoint
import imgaug.augmenters as iaa

from PIL import Image
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import os

## Define hyperparameters:
IMG_SIZE: length and width of the Mantis Images

BATCH_SIZE

EPOCH

NUM_KEYPOINTS: Number of Keypoints to detect. In this work we need to detect 5 Keypoints

In [ ]:
IMG_SIZE = 640
BATCH_SIZE = 10
EPOCHS = 50
NUM_KEYPOINTS = 5 * 2  # 5 pairs each having x and y coordinates

# Predictions on Test Set

get images and annotations of the test set

In [ ]:
IMG_DIR_TEST = r"C:\Users\linus\MantisAI\MantisAI\3_Model\MantisAIData\test"
JSON_TEST = r"C:\Users\linus\MantisAI\MantisAI\3_Model\MantisAIData\MantisTest.json"
KEYPOINT_DEF = r"C:\Users\linus\MantisAI\MantisAI\3_Model\MantisKeypointDef.csv"

# Load the ground-truth annotations.
with open(JSON_TEST) as infile:
    json_data = json.load(infile)

# Prepare the mapping for images and annotations
images = {image['id']: image for image in json_data['images']}
annotations = {annotation['image_id']: annotation for annotation in json_data['annotations']}

# Process keypoints to ensure they are 2D
raw_keypoints = annotations.get('keypoints', [])
if raw_keypoints:
  # Reshape the keypoints into a 2D array with shape (num_keypoints, 3)
  reshaped_keypoints = [raw_keypoints[i:i + 3] for i in range(0, len(raw_keypoints), 3)]
else:
  reshaped_keypoints = []

# Generate the desired output structure
json_dict_test = {}
for image_id, image_data in images.items():
    annotation = annotations.get(image_id, {})

    #convert joints to 2D Array
    keypoints = annotation.get('keypoints', [])
    joints_2d = np.array(keypoints).reshape(-1, 3) if keypoints else []

    # Map the image and annotation data to the desired structure
    json_dict_test[image_data['file_name']] = {
        'img_bbox': annotation.get('bbox', []),
        'img_height': image_data['height'],
        'img_path': image_data['file_name'],
        'img_width': image_data['width'],
        'joints': joints_2d.tolist()
    }

define a function to get the test data


In [ ]:
def get_mantis_test(name):
    """
    Loads the imae and its annotations for a given file name from the JSON data.

    Args:
             name(str) -- The name of the image to load, as listed in json_dict

    Returns:
                dict -- A dictionary with the following information:
                    - img_bbox: The bounding box coordinates for the image
                    - img_height: The height of the image in pixels
                    - img_path: The path to the image file
                    - img_width: The width of the image in pixels
                    - joints: The joint coordinates/Keypoints for the image ([x,y,visibility])
                    - img_data: The image data as a NumPy array
    """
    data = json_dict_test[name]
    img_data = plt.imread(os.path.join(IMG_DIR_TEST, data["img_path"]))
    # If the image is RGBA convert it to RGB.
    if img_data.shape[-1] == 4:
        img_data = img_data.astype(np.uint8)
        img_data = Image.fromarray(img_data)
        img_data = np.array(img_data.convert("RGB"))
    data["img_data"] = img_data

    return data

Prepare test data generator 

In [ ]:
class KeyPointsTestDataset(keras.utils.PyDataset):
    def __init__(self, image_keys, aug, batch_size=BATCH_SIZE, train=False, **kwargs):
        """
        Initializes the KeyPointsDataset for managing and preprocessing test data.

        Args:
            image_keys (list): A list of image identifiers used to access the dataset
            aug (callable): A function or object to apply data augmentation to images and keypoints
            batch_size (int, optional): the number of samples per batch. Defaults to BATCH_SIZE
            train (bool, optional): Indicates whether the dataset is used for Training (True) or Validation/Testing (False). Defaults to False

        Returns:
            Stores images keys, augmentation method, batch size, and mode (train/test)
            Calls the on_epoch_end method to prepare the dataset for the first epoch
        """
        super().__init__(**kwargs)
        self.image_keys = image_keys
        self.aug = aug
        self.batch_size = batch_size
        self.train = train
        self.on_epoch_end()

    def __len__(self):
        """ Calculates the number of batches in the dataset."""
        return len(self.image_keys) // self.batch_size

    def on_epoch_end(self):
        """ Prepares the dataset at the end of each epoch. (Generates a array of indices for the dataset and shuffles them if in training mode)"""
        self.indexes = np.arange(len(self.image_keys))
        # No shuffling for test data
        #if self.train:
        #    np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        """
        Generates one batch of data.

        Args: index(int): Batch index to retrieve
        Returns: tuple (simplified definition: Ordered/sorted form of list): A tuple containing the batch of images and their corresponding keypoints
        """
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        image_keys_temp = [self.image_keys[k] for k in indexes]
        (images, keypoints) = self.__data_generation(image_keys_temp)

        return (images, keypoints)

    def __data_generation(self, image_keys_temp):
        """
        Loads and preprocesses a batch of images and their keypoints.

        Args:
            image_keys_temp (list): A temporary list of image keys to load and preprocess

        Returns:
            tuple:
                - batch_images (np.ndarray): A batch of images
                - batch_keypoints (np.ndarray): A batch of normalized keypoints
        """
        batch_images = np.empty((self.batch_size, IMG_SIZE, IMG_SIZE, 3), dtype="int")
        batch_keypoints = np.empty(
            (self.batch_size, 1, 1, NUM_KEYPOINTS), dtype="float32"
        )

        for i, key in enumerate(image_keys_temp):
            data = get_mantis_test(key) # Assuming get_mantis function is defined elsewhere and handles test data
            current_keypoint = np.array(data["joints"])[:, :2]
            kps = []

            for j in range(0, len(current_keypoint)):
                kps.append(Keypoint(x=current_keypoint[j][0], y=current_keypoint[j][1]))

            current_image = data["img_data"]
            kps_obj = KeypointsOnImage(kps, shape=current_image.shape)

            (new_image, new_kps_obj) = self.aug(image=current_image, keypoints=kps_obj)
            batch_images[i,] = new_image

            kp_temp = []
            for keypoint in new_kps_obj:
                kp_temp.append(np.nan_to_num(keypoint.x))
                kp_temp.append(np.nan_to_num(keypoint.y))

            batch_keypoints[i,] = np.array(kp_temp).reshape(1, 1, 5 * 2)

        batch_keypoints = batch_keypoints / IMG_SIZE

        return (batch_images, batch_keypoints)

create the test dataset

In [ ]:
test_dataset = KeyPointsTestDataset(
    list(json_dict_test.keys()), test_aug, workers=2, use_multiprocessing=True
)

model_test = keras.models.load_model(r"C:\Users\linus\MantisAI\MantisAI\3_Model\MantisAI_TrainedModels\mantisAIUnfreezeMobileNet.keras")
predictions = model_test.predict(test_dataset).reshape(-1, 5, 2) * IMG_SIZE

In [ ]:
# prompt: create a code that save the predicitions with the according image name as well as the ground truth values to a json file

import json

# Assuming 'predictions' is your array of predictions and 'test_dataset' is your test dataset
# and 'test_keys' contains the file names in test dataset

# Create a list to store the results
results = []

# Iterate through the predictions and filenames
for i, pred in enumerate(predictions):
    # Reshape the prediction as done previously (assuming a 5 keypoints setup)
    reshaped_pred = pred.reshape(-1, 5, 2) * IMG_SIZE

    # Get the corresponding image filename
    image_filename = list(json_dict_test.keys())[i]

    # Get the ground truth keypoints
    data = get_mantis_test(image_filename)
    ground_truth_keypoints = np.array(data["joints"])[:, :2] # Extract only x,y coordinates

    # Append to the result list
    results.append({
        "image_name": image_filename,
        "predictions": reshaped_pred.tolist(),  # Convert numpy to list
        "ground_truth": ground_truth_keypoints.tolist()  # Convert numpy to list
    })


# Save the results to a JSON file
with open('/content/drive/MyDrive/MantisModels/predictions.json', 'w') as f:
    json.dump(results, f, indent=4)